<a href="https://colab.research.google.com/github/31indianaSKKU/NBA_CareerLength/blob/main/NCAA_stat_by_Season%26School%26Player.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Jupyter

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin
import time
import os

def get_season_from_page(soup):
    season_element = soup.select_one("#meta div h1 span")
    if season_element:
        return season_element.text.strip().split()[0]
    return ""

def get_school_name(url):
    url_parts = url.split("/")
    return url_parts[-3]


def get_table_data(table):
    header_row = [cell.text.strip() for cell in table.select("thead tr th")]
    data_rows = table.select("tbody tr")
    data = [[cell.text.strip() for cell in row.select("th, td")] for row in data_rows]
    return pd.DataFrame(data, columns=header_row)


for yrs in range(2000,2001):  #season지정
    ratings_url = f"https://www.sports-reference.com/cbb/seasons/men/{yrs}-ratings.html" 
    response = requests.get(ratings_url)
    soup = BeautifulSoup(response.content, "html.parser")
    base_url = "https://www.sports-reference.com/"

    response = requests.get(ratings_url)
    soup = BeautifulSoup(response.content, "html.parser")

    table = soup.find("table", id="ratings")
#     rows = table.select("tbody tr")
    rows = table.select("tbody tr")[:2]  # 첫 5개 행만 선택_test용

    # 빈 데이터프레임 생성
    combined_df = pd.DataFrame()
   # 필요한 컬럼들을 리스트로 저장
    columns_to_drop = ["#", "Class", "Pos", "Height", "Weight", "Hometown", "High School", "RSCI Top 100", "Summary"]

    for row in rows:
        link_element = row.select_one("td:nth-of-type(1) a")
        if link_element:
            link = link_element["href"]
            full_url = base_url + link

            response = requests.get(full_url)
            soup = BeautifulSoup(response.content, "html.parser")

            roster_table = soup.find("table", id="roster")
            per_game_table = soup.find("table", id="per_game")
            totals_table = soup.find("table", id="totals")
            per_min_table = soup.find("table", id="per_min")
            per_poss_table = soup.find("table", id="per_poss")
            advanced_table = soup.find("table", id="advanced")

            data_frames = list()

            if roster_table is not None:
                roster_data = get_table_data(roster_table)
                season = get_season_from_page(soup)
                school = get_school_name(full_url)
                roster_data["Season"] = season
                roster_data["School"] = school
                data_frames.append(roster_data)
            else:
                print(f"{season} {school} Roster table not found.")
                pass

            if per_game_table is not None:
                per_game_data = get_table_data(per_game_table)
                per_game_data = per_game_data.rename(columns=lambda x: x if x in ["Player", "Season", "School"] else x + "_per_game")
                season = get_season_from_page(soup)
                school = get_school_name(full_url)
                per_game_data["Season"] = season
                per_game_data["School"] = school
                data_frames.append(per_game_data)
            else:
                print(f"{season} {school} PerGame table not found.")
                pass

            if totals_table is not None:
                totals_data = get_table_data(totals_table)
                totals_data = totals_data.rename(columns=lambda x: x if x in ["Player", "Season", "School"] else x + "_totals")
                season = get_season_from_page(soup)
                school = get_school_name(full_url)
                totals_data["Season"] = season
                totals_data["School"] = school
                # print("Totals data frame:")
                # print(totals_data.head())
                data_frames.append(totals_data)
            else:
                print(f"{season} {school} Totals table not found.")
                pass

            if per_min_table is not None:
                per_min_data = get_table_data(per_min_table)
                per_min_data = per_min_data.rename(columns=lambda x: x if x in ["Player", "Season", "School"] else x + "_per_min")
                season = get_season_from_page(soup)
                school = get_school_name(full_url)
                per_min_data["Season"] = season
                per_min_data["School"] = school
                # print("Per 40 Min data frame:")
                # print(per_min_data.head())
                data_frames.append(per_min_data)
            else:
                print(f"{season} {school} Per40Min table not found.")
                pass

            if per_poss_table is not None:
                per_poss_data = get_table_data(per_poss_table)
                per_poss_data = per_poss_data.rename(columns=lambda x: x if x in ["Player", "Season", "School"] else x + "_per_poss")
                season = get_season_from_page(soup)
                school = get_school_name(full_url)
                per_poss_data["Season"] = season
                per_poss_data["School"] = school
                # print("Per 100 Poss data frame:")
                # print(per_poss_data.head())
                data_frames.append(per_poss_data)
            else:
                print(f"{season} {school} Per100Poss table not found.")
                pass

            if advanced_table is not None:
                advanced_data = get_table_data(advanced_table)
                advanced_data = advanced_data.rename(columns=lambda x: x if x in ["Player", "Season", "School"] else x + "_advanced")
                season = get_season_from_page(soup)
                school = get_school_name(full_url)
                advanced_data["Season"] = season 
                advanced_data["School"] = school
                # print("Advanced data frame:")
                # print(advanced_data.head())
                data_frames.append(advanced_data)
            else:
                print(f"{season} {school} Advanced table not found.")
                pass

            combined_df = roster_data.copy()
            for col in columns_to_drop:
                try:
                    combined_df = combined_df.drop(col, axis=1)
                except:
                    continue

            for df in data_frames:
                if df is not None:
                    df = df.loc[:, ~df.columns.duplicated()]
                    combined_df = pd.merge(combined_df, df, on=["Player", "Season", "School"], how="outer")

            combined_df = combined_df[["Season", "School"] + [col for col in combined_df.columns if col not in ["Season", "School"]]]
            # Season 값 형식 변경
            # combined_df["Season"] = combined_df["Season"].apply(lambda x: f"{x[:4]}-{x[-2:]}")
            combined_df["Season"] = combined_df["Season"].astype(str)

            # 파일명 생성
            filename = "NCAA_" + season + ".csv"
            #  # 빈 데이터프레임을 CSV 파일로 저장
            # df.to_csv(filename, index=False)

            # 파일이 이미 존재하는 경우
            if filename in os.listdir():
                # 기존 파일 읽어오기
                existing_df = pd.read_csv(filename)
                # 기존 데이터와 현재 데이터 병합
                combined_df = pd.concat([existing_df, combined_df], ignore_index=False)
            # else:
                # combined_df = pd.concat([combined_df] + data_frames, ignore_index=False)

            # CSV 파일로 저장
            combined_df.to_csv(filename, index=False)
            print("Combined DataFrame has been saved as", filename)

            # 일정 시간 대기
            time.sleep(5)  # 5초 대기

print("Crawling complete!")